![Uber Overview](../pics/uber_overview.jpeg)

# Data exploration

This notebook is dedicated to the exploratory data analysis from an open dataset including more than 150 000 rides done with the Uber app in  India in 2024.
We will explore the data quality and we will try to get as many insights as possible through this data using relevant data visualization tools.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# we extract the dataset thanks to pandas and start the exploration part

df = pd.read_csv("../data/rides_data.csv")
df.head(5)

In [ ]:
# getting an overview of the data 

print(df.shape, end ="\n\n")

print(df.describe())

In [ ]:
# we explore the types of each column's data

print(df.dtypes, end="\n\n")

print("Out of the 21 columns we have 9 numerical columns and 12 categorical columns")

In [ ]:
print(df.value_counts("Cancelled Rides by Customer"), end="\n\n")
print(df.value_counts("Reason for cancelling by Customer"), end="\n\n")
print(df.value_counts("Driver Cancellation Reason"), end="\n\n")
print(df.value_counts("Incomplete Rides Reason"), end="\n\n")
print(df.value_counts("Payment Method"), end="\n\n")

In [ ]:
print("Booking status key statistics :\n")

status_counts = df["Booking Status"].value_counts().reset_index()
status_counts.columns = ["Booking Status", "count"]
status_counts["percentage"] = (status_counts["count"] / status_counts["count"].sum() * 100).round(1)
total_row = pd.DataFrame({
    "Booking Status": ["Total"],
    "count": [status_counts["count"].sum()],
    "percentage": [status_counts["percentage"].sum()]})

status_counts = pd.concat([status_counts, total_row], ignore_index=True)

print(status_counts)

In [ ]:
print("Vehicle type key statistics :\n")

vehicle_summary = df.groupby("Vehicle Type").agg(
    Total_Bookings=("Booking ID", "count"),
    Success_Rate=("Booking Status", lambda x: (x.eq("Completed").mean() * 100).round(1)),
    Avg_Distance=("Ride Distance", "mean"),
    Total_Distance=("Ride Distance", "sum")
).reset_index()

# Mise en forme
vehicle_summary["Avg_Distance"] = vehicle_summary["Avg_Distance"].round(2).astype(str) + " km"
vehicle_summary["Total_Distance"] = (vehicle_summary["Total_Distance"] / 1000).round(0).astype(int).astype(str) + "K km"
vehicle_summary["Success_Rate"] = vehicle_summary["Success_Rate"].astype(str) + "%"

print(vehicle_summary)


# Preliminary dataviz

In [ ]:

# Créer une copie avec Success Rate en float
vehicle_summary_plot = df.groupby("Vehicle Type").agg(Total_Bookings=("Booking ID", "count"),
                                                      Success_Rate=("Booking Status", lambda x: (x.eq("Completed").mean() * 100).round(1))).reset_index()

fig, ax1 = plt.subplots(figsize=(10,6))

# Barplot pour Total Bookings
sns.barplot(
    data=vehicle_summary_plot,
    x="Vehicle Type", y="Total_Bookings",
    color="skyblue", ax=ax1
)
ax1.set_ylabel("Booking #", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")

# Axe secondaire pour le Success Rate
ax2 = ax1.twinx()
sns.lineplot(data=vehicle_summary_plot,x="Vehicle Type", y="Success_Rate",color="red", marker="o", linewidth=2, ax=ax2)
ax2.set_ylabel("Success rate (%)", color="red")
ax2.tick_params(axis="y", labelcolor="red")

plt.title("Booking totals vs success rate per vehicle type")
plt.tight_layout()
plt.show()

print("""\nMost bookings come from Auto and Go vehicle types, but their success rates are only average, highlighting operational inefficiencies. 
Premium categories like UberXL and Premier Sedan have lower booking volumes but consistently higher success rates, 
suggesting they could be positioned as reliable, high-value options while efforts should focus on improving completion rates in high-demand categories.""")

In [ ]:
# Let's observe the monthly rides variations

df["Date"] = pd.to_datetime(df["Date"])
df["Month"] = df["Date"].dt.month
month_names = [calendar.month_name[i] for i in range(1, 13)]
df["Day"] = df["Date"].dt.day
df["Weekday"] = df["Date"].dt.weekday
day_names = [calendar.day_name[i] for i in range(7)]
                            
monthly_rides_count = df["Month"].value_counts().sort_index()

plt.bar(monthly_rides_count.sort_index().index, monthly_rides_count.sort_index().values, color="seagreen", width =0.8)
plt.xticks(ticks=monthly_rides_count.index, labels=month_names, rotation=45)

plt.title("Number of rides per month")
plt.xlabel("Month")
plt.ylabel("Ride number")
plt.show()

print("\n\n Monthly rides number are steady throughout the year \n\n")


In [ ]:
# Let's observe the dayly rides variations

weekday_rides_count = df["Weekday"].value_counts().sort_index()

plt.bar(weekday_rides_count.sort_index().index, weekday_rides_count.sort_index().values, color="cornflowerblue", width =0.8)
plt.xticks(ticks=weekday_rides_count.index, labels=day_names, rotation=45)

plt.title("Number of rides per weekday")
plt.xlabel("Day")
plt.ylabel("Ride number")
plt.show()

print("\n\n Same conclusion as for the monthly rides with the weekdays \n\n")

In [ ]:
# Let's observe the hourly rides variations


df["Time"] = pd.to_datetime(df["Time"], format="%H:%M:%S").dt.time
df["Hour"] = pd.to_datetime(df["Time"], format="%H:%M:%S").dt.hour


hourly_rides_count = df["Hour"].value_counts().sort_index()
bars = plt.barh(hourly_rides_count.index, hourly_rides_count.values)

plt.barh(hourly_rides_count.sort_index().index, hourly_rides_count.sort_index().values, color = "yellowgreen")
plt.yticks(ticks=hourly_rides_count.index[::2], labels=hourly_rides_count.index[::2], rotation=45)
plt.title("Number of rides per hour")
plt.xlabel("Ride number")
plt.ylabel("Hour")
plt.bar_label(bars, fmt="%d", label_type = "center", color="black")
plt.show()

print("\n\n We can observe significant variations on rides hours, with a first peak in the morning (10am) \
and a more significant activity between 4pm and 9pm. \n We can also conclude that there is almost no rides between 12am and 5am")


In [ ]:
# We want to know the completed rides shares

bs_counts = df["Booking Status"].value_counts()
colors=["dodgerblue", "royalblue", "teal", "lightseagreen", "cyan"]
plt.pie(bs_counts.values, labels=bs_counts.index, autopct="%.0f%%", colors = colors)

plt.title("Number of rides per booking status")
plt.show()

print("\n\n The main part of rides are completed while 25% of total rides are canceled")

In [ ]:
# Observation of the average time to arrive at trip distribution

plt.hist(df["Avg VTAT"], bins=30, color="skyblue", edgecolor="black")
plt.xlabel("Average VTAT (minutes)")
plt.ylabel("Number of rides")
plt.title("Distribution of average VTAT")
plt.show()

print("\n\n This distribution, not normal, seems coherent : a lot of trips are started immediatly since the driver is near the customer and we can observe \
a few rides that started more than 15 minutes after the customer ordered a trip on the app")

In [ ]:
# Let's have a look at the average waiting time for the driver  once the ride has been booked

fig=px.box(df["Avg CTAT"])
fig.show()

print("\n\n The median time for a customer to arrive to pick up location is almost 29 minutes. Obviously, it seems that this data might include \
reservations made in advance since this waiting time isn't reasonable. We may not use this data later in our analysis")

In [ ]:
# Let's have a look at the cancellation reasons

cust_cancel_values = df["Reason for cancelling by Customer"].value_counts()
fig = px.pie(names = cust_cancel_values.index , values = cust_cancel_values.values, title = "Reasons for cancelling by customer")
fig.show()
print("\n\n The reasons for customer cancellations are diverse and mainly due to personal or situational reasons\n")

driver_cancel_values = df["Driver Cancellation Reason"].value_counts()
fig = px.pie(names = driver_cancel_values.index , values = driver_cancel_values.values, title = "Reasons for cancelling by driver")
fig.show()
print("\n\n The reasons for drivers cancellations are more related to rules and operational constraints\n")

In [ ]:
# Let's try to understand if there is a correlation between ride value and ride distance

sns.scatterplot(data = df, x= "Ride Distance", y="Booking Value")
plt.xlabel("Distance (km)")
plt.ylabel("Booking value (INR)")
plt.title("Booking value vs ride distance")
plt.show()

print(df[["Ride Distance", "Booking Value"]].corr())

print("\n\n Surprisingly, there is almost no correlation between ride distance et ride price")

In [ ]:
# We observe the distribution of ride distance values

hist = px.histogram(df, x="Ride Distance", nbins=15, text_auto=True)

box = px.box(df, y="Ride Distance")

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("Ride distance", "Boxplot of ride distance"))
for trace in hist.data:
    fig.add_trace(trace, row=1, col=1)
for trace in box.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Distance",
    yaxis_title="Number of rides",
    showlegend=False)

fig.show()

print("\n\n The most popular ride distances are between 2.5 and 17.5 km, the median distance , less than 24 km / ride")


In [ ]:
# We observe the distribution of booking values

hist = px.histogram(df, x="Booking Value", nbins=20, text_auto=True, color_discrete_sequence =["forestgreen"])

box = px.box(df, y="Booking Value",color_discrete_sequence =["darkgreen"])

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("Booking Value", "Boxplot of booking value"))
for trace in hist.data:
    fig.add_trace(trace, row=1, col=1)
for trace in box.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(
    bargap=0.2,
    xaxis_title="Booking value",
    yaxis_title="Number of rides",
    showlegend=False)

fig.show()

print("\n\n The median booking value is 414 INR (~5 USD); most rides are cost less than 1,000 INR (11 USD). However, we can observe a long tail \
of extreme values betwwen 1500 and 4000 INR. Since there are a lot of high values and that the highest is less than 50 USD \
, we will not consider them as outliers")

# Data cleaning

In [ ]:
print(df.columns.tolist())

In [ ]:
# Keeping only the useful columns for our analysis

useful_columns=['Month', 
                'Day', 
                'Weekday', 
                'Hour', 
                'Booking ID', 
                'Booking Status', 
                'Customer ID', 
                'Vehicle Type', 
                'Pickup Location', 
                'Drop Location', 
                'Avg VTAT',
                'Reason for cancelling by Customer',
                'Driver Cancellation Reason',
                'Incomplete Rides Reason', 
                'Booking Value', 
                'Ride Distance', 
                'Driver Ratings', 
                'Customer Rating', 
                'Payment Method']

df=df[useful_columns]

print(df.dtypes)
                

In [ ]:
# Renaming the columns to replace spaces and avoid syntax errors later

df = df.rename(columns={
    "Month": "month",
    "Day": "day",
    "Weekday": "weekday",
    "Hour": "hour",
    "Booking ID": "booking_id",
    "Booking Status": "booking_status",
    "Customer ID": "customer_id",
    "Vehicle Type": "vehicle_type",
    "Pickup Location": "pickup_location",
    "Drop Location": "drop_location",
    "Avg VTAT": "avg_vtat",
    "Reason for cancelling by Customer": "reason_for_cancelling_by_customer",
    "Driver Cancellation Reason": "driver_cancellation_reason",
    "Incomplete Rides Reason": "incomplete_rides_reason",
    "Booking Value": "booking_value",
    "Ride Distance": "ride_distance",
    "Driver Ratings": "driver_rating",
    "Customer Rating": "customer_rating",
    "Payment Method": "payment_method"
})

print("Data type per column :\n\n", df.dtypes)



In [ ]:
#Identification of missing value number and rates per column

print("Number of missing values per column :\n\n", df.isna().sum(), "\n\n")

print("Percentage of missing values per column :\n\n", df.isna().mean()*100)

In [ ]:
# Replace the NaN values - by 0 or -1 for numerical variables

df["avg_vtat"] = df["avg_vtat"].fillna(0)
df["booking_value"] = df["booking_value"].fillna(0)
df["ride_distance"] = df["ride_distance"].fillna(0)
df["driver_rating"] = df["driver_rating"].fillna(-1)
df["customer_rating"] = df["customer_rating"].fillna(-1)
df["payment_method"] = df["payment_method"].fillna("No payment")

print("Number of missing values per column :\n\n",df.isna().sum(), "\n\n Now we have a cleaner dataset with no missing values \
except for 3 columns where we should have missing values in case the rides have been completed")

In [ ]:
print("# of duplicated values : ", df.duplicated().sum())

print("\nWe don't have any duplicated entries in this dataset")

In [ ]:
df.set_index("booking_id")

In [ ]:
df.to_csv("../data/rides_data_clean.csv", index= True)
